In [17]:
import numpy as np
import tkinter as tk
from tkinter import messagebox

#checking valid input
def valid_input(entry): 
    try:
        value = int(entry.get())
        if value < 0:
            raise ValueError("Negative value entered.")
        return True
    except ValueError:
        messagebox.showerror("Error", "Either value field is empty or having a negative value")
        return False

def valid_resource_inp(entry):
    try:
        values = [int(x) for x in entry.get().split()]
        if any(value < 0 for value in values):
            raise ValueError("Negative value entered.")
        return True
    except ValueError:
        messagebox.showerror("Error", "Either value field is empty or having a negative value")
        return False

def valid_allocation_inp(entry):
    try:
        rows = entry.get().split("\n")
        for row in rows:
            values = [int(x) for x in row.split()]
            if any(value < 0 for value in values):
                raise ValueError("Negative value entered.")
        return True
    except ValueError:
        messagebox.showerror("Error", "Either value field is empty or having a negative value")
        return False

def valid_request_inp(entry):
    try:
        values = [int(x) for x in entry.get().split()]
        if any(value < 0 for value in values):
            raise ValueError("Negative value entered.")
        return True
    except ValueError:
        messagebox.showerror("Error", "Please enter valid non-negative integers")
        return False

def valid_process_indx(entry, num_processes):
    try:
        index = int(entry.get())
        if index < 0 or index >= num_processes:
            raise ValueError("Invalid process index.")
        return True
    except ValueError:
        messagebox.showerror("Error", f"Please enter a valid process index between 0 and {num_processes - 1}.")
        return False

def safety_algo(available, allocation, need, processes):
    w = available.copy()
    finish = np.zeros(len(processes), dtype=bool)
    safe_seq = []
    
    while len(safe_seq) < len(processes):
        found = False
        for i, process in enumerate(processes):
            if not finish[i] and all(need[i] <= w):
                w += allocation[i]
                finish[i] = True
                safe_seq.append(process)
                found = True
        if not found:
            return None, None
    return safe_seq, need

def resource_request_algo(available, allocation, need, process_index, request, processes):
    if all(request <= need[process_index]) and all(request <= available):
        available -= request
        allocation[process_index] += request
        need[process_index] -= request
        safe_seq, new_need = safety_algo(available, allocation, need, processes)
        if safe_seq is not None:
            return "Request can be granted. Safe sequence: " + ", ".join(safe_seq), new_need
        else:
            available += request
            allocation[process_index] -= request
            need[process_index] += request
            return "Request cannot be granted because it results into an unsafe state.", None
    else:
        return "Request cannot be granted due to insufficient resources or exceeding need.", None


def run_algo():
    if not valid_input(num_resources_entry) or not valid_input(num_processes_entry):
        return

    if not valid_resource_inp(available_entry):
        return

    for entry in allocation_entries:
        if not valid_allocation_inp(entry):
            return

    for entry in max_allocation_entries:
        if not valid_allocation_inp(entry):
            return

    if request_process_entry.get() != "" and not valid_request_inp(request_entry):
        return
        
    num_resources_str = num_resources_entry.get()
    num_processes_str = num_processes_entry.get()
    request_process_str = request_process_entry.get()

    if num_resources_str == "" or num_processes_str == "":
        messagebox.showerror("Error", "Please enter values for all fields.")
        return

    num_resources = int(num_resources_str)
    num_processes = int(num_processes_str)
    
    available_values = available_entry.get().split()
    if len(available_values) != num_resources:
        messagebox.showerror("Error", "Number of values for available resources doesn't match the specified number of resources.")
        return

    available = np.array([int(x) for x in available_values])
    
    allocation = np.zeros((num_processes, num_resources), dtype=int)
    for i in range(num_processes):
        allocation_values = allocation_entries[i].get().split()
        if len(allocation_values) != num_resources:
            messagebox.showerror("Error", f"Number of values for allocation of Process {i} doesn't match the specified number of resources.")
            return
        allocation[i] = [int(x) for x in allocation_values]
    
    max_allocation = np.zeros((num_processes, num_resources), dtype=int)
    for i in range(num_processes):
        max_allocation_values = max_allocation_entries[i].get().split()
        if len(max_allocation_values) != num_resources:
            messagebox.showerror("Error", f"Number of values for max allocation of Process {i} doesn't match the specified number of resources.")
            return
        max_allocation[i] = [int(x) for x in max_allocation_values]
    
    need = max_allocation - allocation
    
    processes = [str(i) for i in range(num_processes)]

    if np.any(allocation > max_allocation):
        messagebox.showerror("Error", "Allocation cannot exceed maximum allocation.")
        return
    
    #running_safety_algo
    safe_seq, need = safety_algo(available, allocation, need, processes)
    if safe_seq is not None:
        safe_seq_label.config(text="Safe sequence: " + ", ".join(safe_seq), fg="green")
    else:
        safe_seq_label.config(text="System is in an unsafe state.", fg="red")
    
    need_label.config(text="Need Matrix:\n" + "\n".join(" ".join(str(x) for x in row) for row in need))

    
     #running_resource_req
    if request_process_str != "" and request_entry.get() != "":
        if not valid_process_indx(request_process_entry, num_processes):
            return
       
        request_process = int(request_process_str)
        request = np.array([int(x) for x in request_entry.get().split()])
        result, need = resource_request_algo(available, allocation, need, request_process, request, processes)
        messagebox.showinfo("Resource Request Result", result)
        
        if need is not None:
            need_label.config(text="Need Matrix:\n" + "\n".join(" ".join(str(x) for x in row) for row in need))

#GUI_part
root = tk.Tk()
root.title("Banker's Algorithm")

bg_color = "#f0f0f0"  
button_color = "#4CAF50"  
label_color = "#333"  
entry_color = "#ddd" 
frame_color = "#ccc"

root.config(bg=bg_color)

inp_frame = tk.Frame(root, bg=frame_color)
inp_frame.pack(padx=30, pady=30)

num_resources_entry = tk.Entry(inp_frame, bg=entry_color)
num_processes_entry = tk.Entry(inp_frame, bg=entry_color)

num_resources_label = tk.Label(inp_frame, text="Number of Resources:", bg=frame_color, fg=label_color)
num_resources_label.grid(row=0, column=0)
num_resources_entry.grid(row=0, column=1)

num_processes_label = tk.Label(inp_frame, text="Number of Processes:", bg=frame_color, fg=label_color)
num_processes_label.grid(row=1, column=0)
num_processes_entry.grid(row=1, column=1)

available_label = tk.Label(inp_frame, text="Available Resources:", bg=frame_color, fg=label_color)
available_label.grid(row=2, column=0)
available_entry = tk.Entry(inp_frame, bg=entry_color)
available_entry.grid(row=2, column=1)

allocation_labels = []
allocation_entries = []

def create_allocation_entries():
    num_processes = int(num_processes_entry.get())
    for i in range(num_processes):
        label = tk.Label(inp_frame, text=f"Allocation for Process {i}:", bg=frame_color, fg=label_color)
        label.grid(row=i+3, column=0)
        allocation_labels.append(label)
        entry = tk.Entry(inp_frame, bg=entry_color)
        entry.grid(row=i+3, column=1)
        allocation_entries.append(entry)

allocation_button = tk.Button(inp_frame, text="Create Allocation Entries", command=create_allocation_entries, bg=button_color, fg="white")
allocation_button.grid(row=3, column=4)

max_allocation_labels = []
max_allocation_entries = []

def create_max_allocation_entries():
    num_processes = int(num_processes_entry.get())
    for i in range(num_processes):
        label = tk.Label(inp_frame, text=f"Max Allocation for Process {i}:", bg=frame_color, fg=label_color)
        label.grid(row=i+3, column=2)
        max_allocation_labels.append(label)
        entry = tk.Entry(inp_frame, bg=entry_color)
        entry.grid(row=i+3, column=3)
        max_allocation_entries.append(entry)

max_allocation_button = tk.Button(inp_frame, text="Create Max Allocation Entries", command=create_max_allocation_entries, bg=button_color, fg="white")
max_allocation_button.grid(row=3, column=5)


need_label = tk.Label(root, text="Need Matrix:", bg=bg_color)
need_label.pack()

run_button = tk.Button(root, text="Run Algorithm", command=run_algo, bg=button_color, fg="white")
run_button.pack(pady=20)

safe_seq_label = tk.Label(root, text="", bg=bg_color)
safe_seq_label.pack()

request_frame = tk.Frame(root, bg=frame_color)
request_frame.pack(pady=20)

request_process_label = tk.Label(request_frame, text="Process Index Requesting Resources:", bg=frame_color, fg=label_color)
request_process_label.grid(row=0, column=0)
request_process_entry = tk.Entry(request_frame, bg=entry_color)
request_process_entry.grid(row=0, column=1)

request_label = tk.Label(request_frame, text="Resource Request:", bg=frame_color, fg=label_color)
request_label.grid(row=1, column=0)
request_entry = tk.Entry(request_frame, bg=entry_color)
request_entry.grid(row=1, column=1)
root.mainloop()